In [1]:
# Получим список файлов, с которыми будем работать

import os
import re
def giveListOfiles(folderName):
    filesList = []
    for dirpath, dirnames, filenames in os.walk(folderName):
        for filename in filenames:
            filesList.append(str(os.path.join(dirpath, filename)))
    for each in filesList:
        if not re.search(r".+[?=txt]", each, flags=re.MULTILINE):
            filesList.remove(each)
    return filesList

Files = giveListOfiles("input")
print(Files)

['input\\bulgakov_m_a__master_i_margarita.txt', 'input\\bulgakov_m_a__sobache_serdce.txt', 'input\\bunin_ivan_zhizn_arseneva.txt', 'input\\gogol_n_v__mertvye_dushi.txt', 'input\\goncharov_i_a__oblomov.txt', 'input\\sholohov_m_a__tihiy_don.txt']


In [2]:
# Поскольку тут для удобства используется findspark, нужно ему указать сам спарк, иначе не заведется

import findspark
findspark.init()

In [3]:
# Создаем Сессию и контекст Спарка

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName('program').getOrCreate()
sc=spark.sparkContext
print("spark version:",spark.version,"sc:",sc)

spark version: 3.3.1 sc: <SparkContext master=local appName=program>


In [4]:
# Читаем итеративно каждый файл. Первый читается как textFile и записываем в базовый. Остальные конкатенируем с первым.

text_file = None
for each in Files:
    if Files.index(each) == 0:
        text_file = sc.textFile(Files[0]) 
        continue
    neText= sc.textFile(each)
    text_file.union(neText).collect()

In [5]:
# посмотрим, что тип RDD. 

print(type(text_file))

<class 'pyspark.rdd.RDD'>


In [6]:
# Разделяем текст на отдельный слова - делителем берем пробел

flatMapped = text_file.flatMap(lambda line: line.split(" "))

In [7]:
# Что мы в итоге сделали

print(type(flatMapped))

<class 'pyspark.rdd.PipelinedRDD'>


In [8]:
# Применяем map функцию. к каждому элементу мы присваиваем значение 1 и делаем пары ключ:значение

mapped = flatMapped.map(lambda word: (word, 1))

In [10]:
# Посмотрим что вышло

mapped.take(10)

[('*******************************************', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1)]

In [11]:
# Вышло пока средне. Строки явно не слова. Для начала уберем в каждой строке лишние символы.

import re
def filterFuncLen(one_line_input):
    checking = one_line_input[0]
    checking = re.sub(r'\-', '', checking, count=0)
    checking = re.sub(r'\ ', '', checking, count=0)
    checking = re.sub(r'\(', '', checking, count=0)
    checking = re.sub(r'\)', '', checking, count=0)
    checking = re.sub(r'\,', '', checking, count=0)
    checking = re.sub(r'\.', '', checking, count=0)
    checking = re.sub(r'\*', '', checking, count=0)
    newRes = [checking,one_line_input[1]]
    # one_line_input[0] = checking
    return [newRes]

mapped2 = mapped.flatMap(filterFuncLen)


In [12]:
# Убрали звездочки

mapped2.take(10)

[['', 1],
 ['', 1],
 ['', 1],
 ['', 1],
 ['', 1],
 ['', 1],
 ['', 1],
 ['', 1],
 ['', 1],
 ['', 1]]

In [13]:
# точно также с помощью flatMap удалим слова, длина которых меньше трех включительно

def filterFuncLen(one_line_input):
        if len(one_line_input[0])>=3:
            return [one_line_input]
        else:
            return []

mapped3 = mapped2.flatMap(filterFuncLen)

In [14]:
# Такой состав слов нас устраивает.

mapped3.take(10)

[['Булгаков', 1],
 ['Текст', 1],
 ['печатается', 1],
 ['последней', 1],
 ['прижизненной', 1],
 ['редакции', 1],
 ['рукопись', 1],
 ['хранится', 1],
 ['рукописном', 1],
 ['деле', 1]]

In [15]:
# проводим reduceByKey. Собираем для каждого ключа сумму значений

reduced = mapped3.reduceByKey(lambda x, y: x + y)

In [16]:
# Отсортируем

sorted1 = reduced.sortBy(lambda x: x[1],ascending=False)

In [17]:
# Посмотрим самые распространенные слова

sorted1.take(20)

[('что', 1656),
 ('как', 749),
 ('его', 635),
 ('это', 518),
 ('было', 444),
 ('все', 369),
 ('она', 306),
 ('был', 294),
 ('так', 286),
 ('маргарита', 281),
 ('уже', 270),
 ('только', 264),
 ('под', 262),
 ('ему', 229),
 ('еще', 210),
 ('ответил', 209),
 ('меня', 205),
 ('этого', 199),
 ('сказал', 198),
 ('того', 196)]

In [18]:
# Отсортируем

sorted2 = reduced.sortBy(lambda x: x[1],ascending=True)

In [19]:
# Посмотрим самые не распространенные слова

sorted2.take(20)

[('Булгаков', 1),
 ('Текст', 1),
 ('печатается', 1),
 ('прижизненной', 1),
 ('хранится', 1),
 ('рукописном', 1),
 ('Государственной', 1),
 ('библиотеки', 1),
 ('СССР', 1),
 ('Ленина', 1),
 ('исправлениями', 1),
 ('полнениями', 1),
 ('поддиктовку', 1),
 ('Булгаковой', 1),
 ('наконец?', 1),
 ('совершает', 1),
 ('Гете', 1),
 ('небывало', 1),
 ('Патриарших', 1),
 ('оде', 1)]

In [20]:
# Принт в цикле

output = sorted1.collect()
i = 100
for (word, count) in output:
    print("{word} - {count}".format(word = word, count = count))
    
    i-=1
    if i < 0 :
        break

что - 1656
как - 749
его - 635
это - 518
было - 444
все - 369
она - 306
был - 294
так - 286
маргарита - 281
уже - 270
только - 264
под - 262
ему - 229
еще - 210
ответил - 209
меня - 205
этого - 199
сказал - 198
того - 196
мне - 194
том - 181
него - 178
чтобы - 172
вас - 170
очень - 167
время - 167
тут - 164
где - 161
нет - 161
глаза - 151
воланд - 150
коровьев - 149
вот - 146
этот - 145
Тут - 139
прокуратор - 139
кот - 137
когда - 133
человек - 130
чем - 130
при - 129
себя - 129
вам - 127
тот - 124
Иван - 122
ничего - 119
быть - 118
раз - 117
над - 117
иван - 115
была - 114
ним - 114
перед - 113
азазелло - 112
Маргарита - 111
этом - 109
были - 108
совершенно - 108
даже - 107
или - 106
если - 106
Это - 106
пилат - 105
чтото - 104
лицо - 104
может - 100
про - 100
они - 99
Так - 97
через - 97
нее - 96
стал - 95
кто - 94
без - 94
будет - 94
теперь - 91
иванович - 91
ней - 88
Она - 86
один - 84
голову - 84
сейчас - 83
там - 83
конечно - 83
себе - 82
будто - 82
гость - 80
Что - 78
вдруг - 78

In [21]:
# Остановим сессию и контекст Спарка
sc.stop()
spark.stop()